### Prompt - 將文本進行NER處理

In [17]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


# 定義 CoT NER Prompt
ner_prompt = PromptTemplate(
    input_variables=["text"],
    template="""
    這是一個命名實體識別（NER）任務，你需要將文本中的實體分類為以下類別：
    ### **命名實體分類方式：**  
    - 人物（Person）：包括個人姓名、稱號、官職、別名等。
    - 時間（Date/Time）：歷史年代、具體年份、月份、日期、時刻等。  
    - 組織（Organization）：政府機構、學術機構、軍事組織、社會團體、企業等。
    - 事件（Event）：戰爭、革命、條約、政策變遷、災難、運動等。
    - 專有名詞（Proper Noun）：包含特定歷史文件、法律條文、計畫名稱等。
    - 數量（Quantity）：具體數字、統計數據、人口數、傷亡數等。
    - 貨幣（Money）：歷史貨幣單位及金額。
    - 比例（Percentage/Ratio）：百分比、比率、分數等。
     
    直接輸出 **JSON **格式，每個句子或事件應該是陣列中的一個獨立 JSON 物件。請勿包含其他文字、步驟或 Markdown 格式標記（例如 ```json）。  
    
    ### **文本輸入範例：愛因斯坦是20世紀最重要的科學家之一，其創立了現代物理學的兩大支柱的相對論及量子力學，有「現代物理學之父」之譽。1933年10月回到美國後，愛因斯坦成為普林斯頓高等研究院的常駐教授。** 
    ### **以下是輸出格式範例：**  
    {{
        "人物": ["愛因斯坦"],
        "時間": ["20世紀"],
        "地點": [],
        "組織": [],
        "專有名詞": ["相對論","量子力學","現代物理學之父"]
        "數量": [],
        "貨幣": [],
        "比例": [],
        "事件": ["創立相對論與量子力學"]
    }},
    {{
        以此類推...
    }}
    ### **範例到此結束。**
    
    將輸入每一段輸入的文本，提取成命名實體：
    {text}
    """
)



### Prompt - 提取 NER實體的關係

In [18]:
# 定義 CoT 關係識別 Prompt
relation_prompt = PromptTemplate(
    input_variables=["entities"],
    template="""
    請分析以下實體，推理它們之間的關係。
    請使用 **逐步推理（Chain-of-Thought, CoT）** 的方式：
    1. 觀察每個實體類別（人物、組織、事件等），並思考它們可能的關聯。
    2. 嘗試基於以下wikidata常見的屬性(Property)推理可能的關係：
        人物關係（Person） 相關屬性：
        - P31 (instance of)：實體所屬的類別，例如：「孫中山」- P31 - 「人類」
        ...略
        組織(Organization)關係相關屬性： 
        - P199 (organizational divisions)：業務部門，本組織的組織部門（不是獨立的法人實體）。
        ...略
        數字概念（Numbber Date/Time）相關屬性：
        - P4876 number of records ：記錄數量，記錄的數量。
        ...略      
        時間概念（Date/Time）相關屬性： 
        - P577 (publication date)：出版日期，作品首次出版或發行的時間。
        ...略       
        事件（Event）相關屬性：
        - P361 (part of)：事件的一部分，例如：「二戰」- P361 - 「太平洋戰爭」
        ...略            
        地理(location/geography)概念相關屬性：
        - P361 (part of)：地點的一部分，例如：「基隆」- P361 - 「台灣」
        ...略
        出版品(publication/book)相關屬性：
        - P1343 (described by source)：記載處，有記載此項的文獻等出處。
        ...略
        
    3. 嘗試根據這些關係屬性，建立關係結構，**主體-關係-客體**。例如：
        - 如果一個人物與事件相關，他可能是 **發起** 或 **參與** 了該事件。
        - 如果一個組織與地點相關，這個組織可能 **位於** 該地點。
        - 如果一個人物與時間相關，這個人物可能 **出生於** 或 **死亡於** 這個時間。

    4. ***請確保輸出結果是標準的 JSON 格式，不包含任何其他文字。***

    格式範例如下:
    輸入實體:
    {{
        "人物": ["愛因斯坦"],
        "時間": ["20世紀"],
        "地點": [],
        "組織": [],
        "專有名詞": ["相對論","量子力學","現代物理學之父"]
        "數量": [],
        "貨幣": [],
        "比例": [],
        "事件": ["創立相對論與量子力學"]
    }},
    {{
        以此類推...
    }}    
    
    輸出關係結構:
    {{
         {{"事件": ["創立相對論及量子力學"]}},{{["主體": "愛因斯坦|人物", "關係": "時間|P585", "客體": "20世紀|時間"],["主體": "愛因斯坦|人物", "關係": "作者|P50", "客體": "相對論|專有名詞"],["主體": "愛因斯坦|人物", "關係": "作者|P50", "客體": "量子力學|專有名詞"]}}
        ,{{"事件": ["移居美國，任職於普林斯頓"]}},{{["主體": "愛因斯坦|人物", "關係": "時間|P585", "客體": "1933年10月|時間"],["主體": "愛因斯坦|人物", "關係": "任職|P108", "客體": "普林斯頓高等研究院|組織"]}}
        以類此類推...
        
    }}
    ### **範例到此結束。**
    
    將輸入實體清單，分析推理後建立成關係結構：
    {entities}
    """
)



In [19]:
import json
import re

# 執行 NER 任務
def extract_entities(text):
    raw_result = ner_chain.run(text)  
    #raw_result = ner_chain.invoke(text)  
    try:
        return json.loads(clean_json_output(raw_result))
    except json.JSONDecodeError:
        return {"error": "NER 解析錯誤", "raw_result": raw_result}

# 執行關係識別任務
def extract_relationships(entities):
    raw_result = relation_chain.run(json.dumps(entities, ensure_ascii=False))
    #raw_result = relation_chain.invoke(json.dumps(entities, ensure_ascii=False))
    try:
        return json.loads(clean_json_output(raw_result))
    except json.JSONDecodeError:
        return {"error": "關係識別解析錯誤", "raw_result": raw_result}

# 清理 JSON 輸出，移除 Markdown 標記
def clean_json_output(raw_output):
    # 移除可能的 Markdown JSON 標記
    cleaned = re.sub(r"```json\s*([\s\S]*?)\s*```", r"\1", raw_output).strip()
    return cleaned

# 安全地解析 JSON 字串
def safe_json_loads(data):
    if isinstance(data, str):
        try:
            return json.loads(data)  # 嘗試解析 JSON 字串
        except json.JSONDecodeError:
            print("錯誤：無法解析 JSON 字串")
            return {}  # 解析失敗時返回空字典
    return data  # 如果本來就是字典，則直接返回

# 將文本進行 NER 和關係識別
def process_text(text):
    # 先執行 NER
    entities = extract_entities(text)
    
    # 如果 NER 解析出錯，直接返回
    if "error" in entities:
        return entities
    
    # 再執行關係識別
    relationships = extract_relationships(entities)
    
    return {
        "entities": entities,
        "relationships": relationships
    }

In [20]:
# import langchain
# # 設定 debug 模式為 False
# #langchain.debug = True
# langchain.debug = False

# print(langchain.__version__)

In [21]:
# 設定基本的 LLM 模型

from langchain_openai import ChatOpenAI
import os

# 設置 LLM 模型
os.environ["GROQ_API_KEY"] = 'gsk_'

llm = ChatOpenAI(
    openai_api_base="https://api.groq.com/openai/v1",
    openai_api_key=os.environ['GROQ_API_KEY'],
    model_name="llama-3.3-70b-versatile",
    #model_name="llama-3.2-3b-preview",        
    #model_name="llama-3.2-3b-preview",
    #model_name="mixtral-8x7b-32768",
    #model_name="llama-3.2-11b-vision-preview",
    #model_name="deepseek-r1-distill-qwen-32b",
    #model_name="deepseek-r1-distill-llama-70b",
    temperature=0.0,
  #  max_tokens=1000,
)

### NER 測試

In [22]:
import os
import json
from pprint import pprint

## NER 測試文本
ner_chain = LLMChain(prompt=ner_prompt, llm=llm)

#test_text = "依據民國32年所發表的開羅宣言，日本應無條件投降並將台灣、澎湖群島歸還中國。因此，國民政府乃於民國33年5月在中央設計局之下設立台灣調查委員會，做為戰後接收台灣之準備。民國34年8月29日，國民政府特任陳儀為台灣省行政長官，負台灣接收及軍政全責；9月7日，政府又任命陳儀兼任台灣省警備總司令。"
#test_text = "光復之初，民眾表現出對國民政府熱烈的歡迎與支持。但由於二次大戰末期，臺灣屢遭美軍轟炸，公共設施遭受相當破壞，戰後生產原料取得不易，技術人才難求，短期內恢復戰前生產水準，自非易事。然而當時中央政府派遣來台灣的官員，集行政、司法、立法、軍事大權於一身的行政長官公署制，非但未能有效復原，臺政卻每況愈下，迭受抨擊，加上溝通不夠與觀念的差異，以及所採取的不良經濟、統治方式與風評不佳的官威軍政，種下釀成日後228事件的遠因。"
test_text = ["「霧社事件」發生在民國19年，由霧社地區德哥塔雅原住民賽德克的泰雅人揭竿起義，反抗日本帝國主義侵害行為之不幸事件，依據目睹該事件悲慘景象之巫金墩所述(詳下列檔案影像)，該事件係當時在霧社公校舉行例行運動大會時，由莫那魯道酋長率領部落勇士，全副武裝衝入運動場，殺死運動場上約一百六十餘個日本人，導致台灣總督府採取強力的軍事行動，使用國際所禁用的方式派飛機投擲化學毒氣，以慘無人道的手法殘害原住民。除事件領導人莫那魯道自縊外，該族族民亦幾遭滅族。",
    "巫金墩指出，霧社事件係起因於當時的日本完全忽視泰雅族的習慣，除屢次徵召原住民義務勞動苦役、欺壓、虐待外，日本警官亦凌辱部落婦女，加上「以蕃制蕃」措施，長期以來的種族衝突，讓原住民在無法忍受的怨恨累積下，點燃其反日情節，終致霧社事件的發生。",
    "當時的台灣總督府不但無法迅速處理因應，甚至採取不人道的手段鎮壓，因而遭到日本帝國議會強烈質疑，總督石塚英藏與總務長官人見次郎遭到撤換，嚴重影響軍國主義天皇思想的威信，總督府高壓式的「理蕃」政策也因此被迫做適度的調整修正；霧社事件當可說是日本統治台灣期間最後一次激烈的台灣反抗行動。",
    "電影是人類文化的新興載體，可直接記錄與再現人類生活歷史及自然界種種現象之原貌，透過電影手法將歷史檔案普及化的模式，正是導演籌拍史詩電影，將原住民抗日精神加以重現的另一種典範，令人期待之餘，我們已先從國家檔案一窺端倪囉。"
]
ner = ner_chain.invoke({"text": test_text})
#print(response)
#print(json.dumps(response, indent=4, ensure_ascii=False))
#print(json.dumps(response, indent=2, ensure_ascii=False))
pprint(ner, indent=4, width=80, compact=False)

{   'text': '{\n'
            '    "人物": ["莫那魯道", "巫金墩", "石塚英藏", "人見次郎"],\n'
            '    "時間": ["民國19年"],\n'
            '    "組織": ["日本帝國", "台灣總督府"],\n'
            '    "專有名詞": ["霧社事件", "泰雅人", "賽德克", "霧社公校"],\n'
            '    "數量": ["一百六十餘個"],\n'
            '    "貨幣": [],\n'
            '    "比例": [],\n'
            '    "事件": ["霧社事件", "運動大會", "軍事行動", "抗日行動"]\n'
            '},\n'
            '{\n'
            '    "人物": ["巫金墩", "莫那魯道"],\n'
            '    "時間": [],\n'
            '    "組織": ["日本帝國", "台灣總督府"],\n'
            '    "專有名詞": ["霧社事件", "泰雅族", "以蕃制蕃"],\n'
            '    "數量": [],\n'
            '    "貨幣": [],\n'
            '    "比例": [],\n'
            '    "事件": ["霧社事件", "種族衝突"]\n'
            '},\n'
            '{\n'
            '    "人物": ["石塚英藏", "人見次郎"],\n'
            '    "時間": [],\n'
            '    "組織": ["日本帝國", "台灣總督府", "日本帝國議會"],\n'
            '    "專有名詞": ["霧社事件", "理蕃政策"],\n'
            '    "數量": [],\n'
            '    "貨幣": [],\n'
           

### RE 測試

In [23]:
# 建立關係識別 LLMChain
relation_chain = LLMChain(prompt=relation_prompt, llm=llm)
# 再執行關係識別
relationships = extract_relationships(ner)

pprint(relationships, indent=4, width=80, compact=False)

[   {   '事件': ['霧社事件', '運動大會', '軍事行動', '抗日行動'],
        '關係': [   {'主體': '莫那魯道|人物', '客體': '霧社事件|事件', '關係': '參與|P1344'},
                  {'主體': '巫金墩|人物', '客體': '霧社事件|事件', '關係': '參與|P1344'},
                  {'主體': '石塚英藏|人物', '客體': '霧社事件|事件', '關係': '參與|P1344'},
                  {'主體': '人見次郎|人物', '客體': '霧社事件|事件', '關係': '參與|P1344'},
                  {'主體': '霧社事件|事件', '客體': '民國19年|時間', '關係': '發生於|P585'},
                  {'主體': '霧社事件|事件', '客體': '日本帝國|組織', '關係': '涉及|P361'},
                  {'主體': '霧社事件|事件', '客體': '台灣總督府|組織', '關係': '涉及|P361'},
                  {'主體': '霧社事件|事件', '客體': '泰雅人|專有名詞', '關係': '涉及|P361'},
                  {'主體': '霧社事件|事件', '客體': '賽德克|專有名詞', '關係': '涉及|P361'},
                  {'主體': '霧社事件|事件', '客體': '霧社公校|專有名詞', '關係': '涉及|P361'}]},
    {   '事件': ['霧社事件', '種族衝突'],
        '關係': [   {'主體': '巫金墩|人物', '客體': '霧社事件|事件', '關係': '參與|P1344'},
                  {'主體': '莫那魯道|人物', '客體': '霧社事件|事件', '關係': '參與|P1344'},
                  {'主體': '霧社事件|事件', '客體': '日本帝國|組織', '關

### 顯示已處理檔案內容

In [24]:
import os
import json
out_dir = "./docs/output/3_ner_re_p/llama_v7/"
json_dir = out_dir  # JSON檔案所在資料夾

# 取得所有 JSON 檔案
files = [f for f in os.listdir(json_dir) if f.endswith(".json")]

for file in files:
    full_path = os.path.join(json_dir, file)
    
    with open(full_path, "r", encoding="utf-8") as f:
        data = json.load(f)
        
        print(f"\n===== 檔案名稱: {file} =====")
        print(json.dumps(data, indent=2, ensure_ascii=False))


===== 檔案名稱: 228事件(20).json =====
{
  "result": {
    "entities": [
      {
        "人物": [],
        "時間": [
          "光復之初",
          "二次大戰末期"
        ],
        "組織": [
          "國民政府"
        ],
        "專有名詞": [],
        "數量": [],
        "貨幣": [],
        "比例": [],
        "事件": [
          "二次大戰",
          "228事件"
        ],
        "地點": [
          "臺灣"
        ]
      },
      {
        "人物": [
          "葉德根",
          "傅學通",
          "林江邁",
          "陳文溪"
        ],
        "時間": [
          "民國36年2月27日"
        ],
        "組織": [
          "臺灣省菸酒專賣局",
          "警察大隊"
        ],
        "專有名詞": [
          "天馬茶房"
        ],
        "數量": [
          "6名",
          "4名",
          "5箱",
          "40歲"
        ],
        "貨幣": [],
        "比例": [],
        "事件": [
          "查緝",
          "走私"
        ],
        "地點": [
          "淡水",
          "臺北市",
          "延平北路"
        ]
      },
      {
        "人物": [
          "陳文溪"
        ],
        "時間": [
          

### 處理目錄裡面檔案

In [25]:
import json
import os
import shutil


json_dir = "./docs/output/1_clean_json/llama/"
done_dir = "./docs/output/1_clean_json/llama/done/"
out_dir = "./docs/output/3_ner_re_p/llama_v7/"

# 確保 json_done 目錄存在
#os.makedirs(done_dir, exist_ok=True)
#os.makedirs(out_dir, exist_ok=True)


json_files = [f for f in os.listdir(json_dir) if f.endswith(".json")]

for filename in json_files:
    file_path = os.path.join(json_dir, filename)
    
    try:
        # 讀取 JSON 檔案
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)
        print(f"✅ 開始處理: {filename}")
        # 取得 articles 欄位並進行處理
        articles = data.get("articles", [])

        processed_articles = process_text(articles)
        
        # 將處理後的結果寫回 JSON
        output_data = {
            #輸出加入articles內容
            #"articles": articles,
            "result": processed_articles
        }
        
        out_path = os.path.join(out_dir, filename)
        
        with open(out_path, "w", encoding="utf-8") as f:
            json.dump(output_data, f, indent=2, ensure_ascii=False)
        
        print(f"✅ 已處理並存入: {out_path}")
        
        # 搬移已處理的 JSON 檔案到 json_done 目錄
        shutil.move(file_path, os.path.join(done_dir, filename))
        print(f"📂 已搬移 {filename} 到 {done_dir}")
        
    except json.JSONDecodeError:
        print(f"⚠️ JSON 解析錯誤: {filename}")
    except IOError as e:
        print(f"⚠️ 無法讀取檔案 {filename}，錯誤: {e}")
    print("-" * 50)
    
print("\n📂 全部 JSON 檔案處理完成！")

✅ 開始處理: 九年國教：春風化雨五十載(134).json
✅ 已處理並存入: ./docs/output/3_ner_re_p/llama_v7/九年國教：春風化雨五十載(134).json
📂 已搬移 九年國教：春風化雨五十載(134).json 到 ./docs/output/1_clean_json/llama/done/
--------------------------------------------------
✅ 開始處理: 亞東關係協會：臺日友好關係的樞紐(185).json
✅ 已處理並存入: ./docs/output/3_ner_re_p/llama_v7/亞東關係協會：臺日友好關係的樞紐(185).json
📂 已搬移 亞東關係協會：臺日友好關係的樞紐(185).json 到 ./docs/output/1_clean_json/llama/done/
--------------------------------------------------
✅ 開始處理: 人民頭家—公民直選總統(23).json


KeyboardInterrupt: 